In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
wd = %pwd #This saves the working directory you have set for later use..
import os
import sys
fdel = os.path.sep
sys.path.append(wd)       #This adds the working directory to the environment path variable so that you can import the custom module "profilometry"
import profilometry as pf           #This is the custom module I have written for profilometry data. 
# %load_ext autoreload      #This is an IPython magic command that reloads the custom module automatically so that if you change some code there
# %autoreload 2           #You won't have to close and repoen this .ipynb file.  You only have to run it once per session. 
    

In [2]:
### The function.measure_thickness() function is used to measure the thickness of a sample, it brings up a dialog that let's you choose the.csv file
#containing your raw data and then it will plot the data and follow the instructions printed at top right of chart. 
# This returns a tuple with [0] index being a float of your height measurement and 
# the .trace_profile object (at index [1]) that contains your loaded data and everything to do with that measuremnt you just ran. 
thick_measure = pf.main.measure_thickness()



 proflometry module by James Bishop, use for analysis of profilometry trace data from DekTak profilomoter. Email: james.bishop@uts.edu.au if you have further questions. 


Adding vertical line for measurement at x =  0.5690098309546116
Adding vertical line for measurement at x =  1.1933178340539663
Will fit substrate baseline for leveling between, x1: 0.5690098309546116, x2: 1.1933178340539663
X1 and X2 set:  0.5690098309546116 1.1933178340539663
Fitting line to substrate between x1 and x2 using linear regression (np.polyfit(deg = 1)).
          Is a LBF with equal weighting to all points between x1 & x2.
Adding vertical line for measurement at x =  0.009865289458219795
Will fit substrate baseline for leveling between, x1: 0.5690098309546116, x2: 1.1933178340539663


In [35]:
def clip_via_der_thresh(series: pd.Series, pos_threshold: float, neg_threshold: float) -> pd.Series:
    # Initialize a mask with all 0 values
    mask = pd.Series(0, index=series.index)
    
    # Assign 1 for values above the positive threshold
    mask[series > pos_threshold] = 1
    
    # Assign -1 for values below the negative threshold
    mask[series < neg_threshold] = -1
    
    return mask

def find_ranges(series: pd.Series) -> list:
    ranges = []
    start = None
    sign = 0

    for i in range(len(series)):
        if start is None:
            if series.iloc[i] != 0:
                start = i
                sign = series.iloc[i]
        elif series.iloc[i] == -sign:
            if i + 1 < len(series) and series.iloc[i + 1] == 0:
                ranges.append((start, i))
                start = None
                sign = 0

    # Handle case where series ends without closing the range
    if start is not None:
        ranges.append((start, len(series) - 1))

    return ranges

shifted = thick_measure[1].data.shift().rename('Shifted')
der = thick_measure[1].data.diff().rename('Derivative')
dat = pd.concat([thick_measure[1].data, shifted, der], axis=1) #.drop(thick_measure[1].data.index[0]),
mask = clip_via_der_thresh(der, 2.5, -2.5)
dat['mask'] = mask
ax = dat['mask'].plot()

# # Find ranges to remove
ranges = find_ranges(dat['mask'])
print(ranges)
# for i in range(len(ranges)):
#     ax.axvspan(ranges[i][0], ranges[i][1], color='red', alpha=0.5)
# #pd.Series(ranges, index = der.index).plot()
plt.show()

[(119, 302), (304, 1183), (1185, 3339), (3343, 3376), (3378, 3772), (3775, 6148), (6150, 6921), (6923, 7712), (7715, 7920), (7922, 8999)]


In [58]:
def clip_der_thresh_abs(series: pd.Series, threshold: float, index_postion_distance: int) -> pd.Series:
    der = series.diff().rename('Derivative')
    mask = pd.Series(0, index=series.index)
    mask[der.abs() > threshold] = 1

    grouped_mask = mask.copy().reset_index(drop=True)
    non_zero_positions = grouped_mask[grouped_mask != 0].index
    print(non_zero_positions)
    for i in range(len(non_zero_positions) - 1):
        if non_zero_positions[i + 1] - non_zero_positions[i] <= index_postion_distance:
            grouped_mask.iloc[non_zero_positions[i]:non_zero_positions[i + 1] + 1] = 1
    return pd.Series(grouped_mask.to_list(), index=series.index)

mask = clip_der_thresh_abs(dat["Total Profile(nm)"], 1, 10)
ax = mask.plot()
dat["Despiked"] = dat["Total Profile(nm)"].copy()
dat["Despiked"].loc[mask == 1] = np.nan

ax2 = dat["Despiked"].plot()

plt.show()

Index([   2,  112,  113,  114,  115,  116,  119,  120,  294,  295,
       ...
       7920, 7922, 7923, 7924, 7925, 7926, 7927, 7928, 8083, 8084],
      dtype='int64', length=255)


/tmp/ipykernel_19382/3726969579.py:17: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dat["Despiked"].loc[mask == 1] = np.nan
/tmp/ipykernel_19382/3726969579.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [63]:
dat['EWMA_FB'] = pf.main.ewma_fb(dat['Despiked'], 10)
ax = dat['Despiked'].plot()
ax.plot(dat['EWMA_FB'], label='EWMA_FB', lw = 1, alpha = 0.6, color = 'red')
plt.show()

In [50]:
dat['Total Profile(nm)_f'] = pf.main.cssplinefill(dat['Total Profile(nm)'])
dat['Total Profile(nm)_f'].plot()
plt.show()